This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [33]:
# First, import the relevant modules
import requests
import json

In [34]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r= requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?collapse=daily&rows=2&api_key=")

In [35]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.status_code)
print(r.headers['content-type'])
print(r.headers['X-RateLimit-Limit'])
print(r.text)


200
application/json; charset=utf-8
50000
{"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.\u003cbr\u003e\u003cbr\u003eTrading System: Xetra\u003cbr\u003e\u003cbr\u003eISIN: DE0005313704","refreshed_at":"2019-04-22T22:35:59.509Z","newest_available_date":"2019-04-18","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":2,"transform":null,"column_index":null,"start_date":"2000-06-07","end_date":"2019-04-18","data":[["2019-04-18",null,80.85,79.7,80.55,null,139210.0,11191903.0,null,null,null],["2019-04-17",null,81.5,79.35,80.6,null,184162.0,14795080.0,null,null,null]],"collapse":"daily","order":null,"database_id":6129}}
{'dataset': {'id': 10095370,

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [164]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
#2. Convert the returned JSON object into a Python dictionary.
data_dict = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=tKBVsSzbDnpjuEwDZJno").json()

#3. Calculate what the highest and lowest opening prices were for the stock in this period.
def column(matrix, i):
    #return [row[i] for row in matrix]
    new_list = []
    for row in matrix:
        if row[i] is None:
            val = 0
        else:
            val = row[i]
        new_list.append(val)    
    return new_list    

column_list = data_dict['dataset']['column_names']
high_idx = column_list.index('High')
low_idx = column_list.index('Low')
open_idx = column_list.index('Open')
tradeVol_idx = column_list.index('Traded Volume')
close_idx = column_list.index('Close')
print('The highest opening price for the stock in this period was $', max(column(data_dict['dataset']['data'], open_idx)))
print('The lowest opening price for the stock in this period was $', min(column(data_dict['dataset']['data'], open_idx)))


#4. What was the largest change in any one day (based on High and Low price)?
high_list = column(data_dict['dataset']['data'], high_idx)
low_list = column(data_dict['dataset']['data'], low_idx)
diff = (high_val - low_val for high_val, low_val in zip(high_list, low_list))
print('the largest change in any one day is $' , max(diff))


#5. What was the largest change between any two days (based on Closing Price)?
change = 0
close_list = column(data_dict['dataset']['data'], close_idx)
for i in range(len(close_list)):
    j=i+1
    while j < len(close_list):
        if change < abs(close_list[i]-close_list[j]):
            change = abs(close_list[i]-close_list[j])
        j = j+1        
print ('The largest change between any two days', change)        


#6. What was the average daily trading volume during this year?
def mean(numbers):
    return float(sum(numbers))/ len(numbers)
tradeVol_list = column(data_dict['dataset']['data'], tradeVol_idx)
print('The average daily trading volume during this year was ', mean(tradeVol_list))


#7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
def median(numbers):
    tradeVol_list.sort()
    if (len(tradeVol_list)%2 == 0):
        medianVal = (tradeVol_list[len(tradeVol_list)/2] + tradeVol_list[len(tradeVol_list)/2 -1]) / 2
    else:
        medianVal = tradeVol_list[int(len(tradeVol_list)/2)]
    return medianVal
print('The median trading volume during this year was ' , median(tradeVol_list))



The highest opening price for the stock in this period was $ 53.11
The lowest opening price for the stock in this period was $ 0
the largest change in any one day is $ 2.8100000000000023
The largest change between any two days 19.03
The average daily trading volume during this year was  89124.33725490196
The median trading volume during this year was  76286.0
